Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [ ]:
import numpy as np
from dataclasses import dataclass
from typing import Optional, Tuple
from icecream import ic
from tqdm.auto import tqdm

In [3]:
def inspect_solution(solution):

    for k in range(NUM_KNAPSACKS):
        mask = solution == k
        items = np.where(mask)[0].tolist()
        if len(items) == 0:
            continue
        weights_sum = WEIGHTS[mask].sum(axis=0)

        if (weights_sum > CONSTRAINTS[k]).any():
            return False
    
    return True

Source / Inspiration:
Greedy start solution inspired by "Multidimensional knapsack problem" — Wikipedia
https://en.wikipedia.org/wiki/Multidimensional_knapsack_problem

Implementation adapted for this notebook.

In [4]:

@dataclass
class State:
    assign: np.ndarray   
    loads: np.ndarray    
    value: int           

def initial_empty() -> State:
    return State(
        assign=np.full(NUM_ITEMS, -1, dtype=np.int32),
        loads=np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS), dtype=np.int64),
        value=0
    )

def greedy_fill(rng: np.random.Generator) -> State:
    
    st = initial_empty()
    score = VALUES / (1.0 + WEIGHTS.mean(axis=1))
    for i in np.argsort(-score):
        for k in rng.permutation(NUM_KNAPSACKS):
            new_loads = st.loads[k] + WEIGHTS[i]
            if np.all(new_loads <= CONSTRAINTS[k]):
                st.assign[i] = k
                st.loads[k] = new_loads
                st.value += int(VALUES[i])
                break
    return st


def feasible_move(st: State, item: int, dest: int) -> bool:
    
    cur = st.assign[item]
    if cur == dest:
        return True
    if dest >= 0:
        after = st.loads[dest] + WEIGHTS[item]
        if np.any(after > CONSTRAINTS[dest]):
            return False
    return True

def apply_move(st: State, item: int, dest: int):
    cur = st.assign[item]
    if cur == dest:
        return
    w = WEIGHTS[item]
    if cur >= 0:
        st.loads[cur] -= w
        st.value -= int(VALUES[item])
    if dest >= 0:
        st.loads[dest] += w
        st.value += int(VALUES[item])
    st.assign[item] = dest

def random_feasible_move(st: State, rng: np.random.Generator, max_tries: int = 50) -> Optional[Tuple[int, int, int]]:
    
    for _ in range(max_tries):
        i = int(rng.integers(0, NUM_ITEMS))
        dest = int(rng.integers(-1, NUM_KNAPSACKS))
        if not feasible_move(st, i, dest):
            continue
        cur_val = VALUES[i] if st.assign[i] >= 0 else 0
        new_val = VALUES[i] if dest >= 0 else 0
        delta = int(new_val - cur_val)
        return (delta, i, dest)
    return None


def hill_climbing(rng: np.random.Generator, start: Optional[State] = None, steps: int = 1500, max_tries: int = 50):
    st = start if start is not None else greedy_fill(rng)
    
    for _ in tqdm(range(steps)):
        mv = random_feasible_move(st, rng, max_tries=max_tries)
        if mv is None: continue
        delta, i, dest = mv
        if delta >= 0:
            apply_move(st, i, dest)
    return st

## TEST PROBLEMS

In [5]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

start = greedy_fill(np.random.default_rng())
best = hill_climbing(np.random.default_rng(), start=start, steps=1000, max_tries=50)
assigned = int((best.assign >= 0).sum())
print(f"Valore={best.value}  Assegnati={assigned}  Non assegnati={NUM_ITEMS - assigned}")
valid = inspect_solution(best.assign)
ic(valid)

NameError: name 'tqdm' is not defined

In [ ]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

start = greedy_fill(np.random.default_rng())
best = hill_climbing(np.random.default_rng(), start=start, steps=20000, max_tries=50)
assigned = int((best.assign >= 0).sum())
print(f"Valore={best.value}  Assegnati={assigned}  Non assegnati={NUM_ITEMS - assigned}")
valid = inspect_solution(best.assign)
ic(valid)

  0%|          | 0/20000 [00:00<?, ?it/s]

ic| valid: True


Valore=46504  Assegnati=74  Non assegnati=26


True

In [ ]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)
start = greedy_fill(np.random.default_rng())
best = hill_climbing(np.random.default_rng(), start=start, steps=50000, max_tries=50)
assigned = int((best.assign >= 0).sum())
print(f"Valore={best.value}  Assegnati={assigned}  Non assegnati={NUM_ITEMS - assigned}")
valid = inspect_solution(best.assign)
ic(valid)

  0%|          | 0/50000 [00:00<?, ?it/s]

ic| valid: True


Valore=1884322  Assegnati=2748  Non assegnati=2252


True